In [ ]:
# We are loading our Document from Directory
# We are using DirectoryLoader class provided by Langchain
from langchain.document_loaders import DirectoryLoader
directory='B:/Major_Git/LangChain models/Chatbot_Answering_from_Your_Own_Knowledge_Base/book'
def load_docs(directory):
    loader=DirectoryLoader(directory)
    documents=loader.load()
    return documents
documents=load_docs(directory)
len(documents)

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


1

In [17]:
# We will split this document into smaller chunks 
from langchain.text_splitter import RecursiveCharacterTextSplitter
def split_docs(documents,chunk_size=500,chunk_overlap=20): # Chunk overlap will decide how much text from previous chunk is retained in the next chunk
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    docs=text_splitter.split_documents(documents)
    return docs
docs=split_docs(documents)
print(len(docs))

496


In [19]:
# We will create embeddings of that created chunk of document.
# convert text into numerical format .So our model can understand
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings=SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')

C:\Users\ombha\AppData\Local\Temp\ipykernel_16468\2145657351.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')
b:\Major_Git\LangChain models\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
''' We are going to store this Embeddings into our Database- Pinecone'''
import os
import pinecone
from pinecone import Pinecone ,ServerlessSpec ,ServerlessSpec, CloudProvider, AwsRegion,VectorType
from langchain.vectorstores import Pinecone as PineconeStores

pc=Pinecone(api_key='PINECONE_API_KEY',environment="us-east-1")
index='langchain-chatbot'
pc.create_index(
    name=index,
    dimension=384,
    metric='cosine',
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)


{
    "name": "langchain-chatbot",
    "metric": "cosine",
    "host": "langchain-chatbot-gru85pp.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [50]:
from langchain_pinecone import PineconeVectorStore
from dotenv import load_dotenv
load_dotenv()
index=PineconeVectorStore.from_documents(docs,embeddings,index_name=index)

In [51]:
# Checking the Similiraty of Question
def get_similar_docs(query,k=1,score=False):
    if score:
        similar_docs=index.similarity_search_with_score(query,k=k)
    else:
        similar_docs=index.similarity_search(query,k=k)
    return similar_docs

In [52]:
query='Tell me about India'
similar_docs=get_similar_docs(query)
similar_docs

[Document(id='688f1c31-d4fa-4b93-8ba6-44bdf1a896d6', metadata={'source': 'B:\\Major_Git\\LangChain models\\Chatbot_Answering_from_Your_Own_Knowledge_Base\\book\\india.txt'}, page_content="India's economy\n\nis estimated to have been the largestin the world, having between one-third and one-quarter of the world's wealth, from 1 CE to 1000CE.\n\n[\n\n127\n\n]\n\n[\n\n128\n\n]\n\nClassical period (c. 200 BCE – 650 CE)\n\n26/03/2025, 14:17 History of India - Wikipedia\n\nhttps://en.wikipedia.org/wiki/History_of_India 12/75\n\nShunga Empire\n\nEast Gateway and Railings,\n\nBharhut\n\nStupa\n\n, 2nd century BCE.\n\nShunga art\n\nAncient vina\n\n, 1stcentury BCE.\n\nRoyal family, 1stcentury BCE inWest Bengal.")]